In [1]:
import numpy as np
import numpy.random as rand
from scipy.constants import codata
import scipy.interpolate as si
from todloop.utils.pixels import PixelReader
from todloop.base import Routine, TODLoop
import cPickle
import os
import pprint
from todloop.utils.cuts import pixels_affected_in_event
from plot_glitches.routines import PlotGlitches
import moby2
from moby2.scripting import get_filebase
from random import *
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/mnt/act3/users/jpm136/miniconda2/lib/python2.7/site-packages/moby2/util/noninteractive_plots.py:2: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'TkAgg' by the following code:
  File "/mnt/act3/users/jpm136/miniconda2/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/mnt/act3/users/jpm136/miniconda2/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/mnt/act3/users/jpm136/miniconda2/lib/python2.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/mnt/act3/users/jpm136/miniconda2/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/mnt/act3/users/jpm136/miniconda2/lib/python2.

In [33]:
def MonteCarlo_double(f, g, x_olim, y_olim, x_lim, y_lim, n):
    """
    #-------------------------------------------------#
    # Description:                                    #
    # Monte Carlo integration of f over a domain g>=0,#
    # n^2 is the number of random points.             #
    #                                                 #
    # Parameters:                                     #
    # f: integrand function                           #
    # g: function that decides which points are       #
    #    within bounds                                #
    # x_olim, y_olim: lists of overshot bounds of     #
    #                 integration                     #
    # x_lim, y_lim: lists of actual bounds of         #
    #               integration                       #
    # n: number of points to define intgration grid   #
    #-------------------------------------------------#
    """
    #Unpack Variables
    x0,x1 = x_olim[0],x_olim[1]
    y0,y1 = y_olim[0],y_olim[1]
    x_0,x_1 = x_lim[0],x_lim[1]
    y_0,y_1 = y_lim[0],y_lim[1]
    
    # Draw n**2 random points in the area of integration  
    x = np.random.uniform(x0, x1, n)
    y = np.random.uniform(y0, y1, n)
    
    # Compute sum of f values inside the integration domain
    f_mean = 0
    num_inside = 0   # number of x,y points inside domain (g>=0)
    for i in range(len(x)):
        for j in range(len(y)):
            if g(x[i], y[j],x_lim,y_lim) >= 0:
                num_inside += 1
                f_mean += f(x[i], y[j])
    f_mean = f_mean/float(num_inside)
    area = num_inside/float(n**2)*(x1 - x0)*(y1 - y0)
    return area*f_mean

def g(x, y,x_lim,y_lim):
    
    """
    x_lim: list of starting and ending bounds of x-integration
    y_lim: list of starting and ending bounds of y-integration
    """
    x0,x1 = x_lim[0],x_lim[1]
    y0,y1 = y_lim[0],y_lim[1]
    return(1 if (x0 <= x <= x1 and y0 <= y <= y1) else -1)
    
print MonteCarlo_double(lambda x, y: np.sin(x * y**2),g, [0,1.5*np.pi],[0,1.5*np.pi],[0,np.pi],[0,np.pi],100)



1.48180811026833


In [ ]:
tod = 10000
input_dir="../outputs/s16_pa3_list/cosig/"
postfix = "pickle"
filepath = "%s%s.%s" % (input_dir, tod, postfix)
if os.path.isfile(filepath):
    with open(filepath, "r") as f:
        cuts = cPickle.load(f)
else:
    print 'Cosig data NOT loaded!'

fb = get_filebase()
loop = TODLoop()
loop.add_tod_list("../data/s16_pa3_list.txt")
loop.run(tod,tod+1)
tod_name = loop.get_name()
tod_filename = fb.filename_from_name(tod_name, single=True)  # get file path
tod_data = moby2.scripting.get_tod({'filename': tod_filename, 'repair_pointing': True})
amdata = moby2.scripting.get_array_data({'season' : '2016','array_name': 'AR3'})
sky_x = amdata['sky_x']
sky_y = amdata['sky_y']

#fix optical sign
optical_signs = tod_data.info.array_data['optical_sign']
tod_data.data = tod_data.data*optical_signs[:, np.newaxis]

#calibrate
cal = moby2.scripting.get_calibration({'type': 'iv', 'source': 'data'}, tod=tod_data)
cal_mask, cal_val = cal.get_property('cal', det_uid=tod_data.det_uid)
tod_data.data *= cal_val[:,None]
    
#print(cuts['coincident_signals'])
#cosig = cPickle.load(open(str(tod)+".pickle", "r"))
#cosig = cPickle.load(open("1111.pickle", "r"))

In [ ]:
Beam_PA3_90 = np.loadtxt('beam_profile_170112_2015_pa3_90GHz_instant.txt', unpack=True, delimiter='  ', skiprows=4)

pa3_90x = Beam_PA3_90[0]
pa3_90y = Beam_PA3_90[1]
x = np.linspace(min(pa3_90x),max(pa3_90x),1000)
beam_profile = si.interp1d(pa3_90x,pa3_90y)

plt.figure(figsize=(12,8))
plt.title('Beam Profile of PA3 90 GHz')
plt.plot(pa3_90x,pa3_90y,'r',lw=2,label="raw")
plt.plot(x,beam_profile(x),'k--',label="interp")
plt.legend()


In [ ]:
def Brightness_Temperature(Signal, Freq, Omega):
    """
    #------------------------------------------------#
    # Description: Computes the Brightness T in      #
    #              Kelvin for an FRB signal seen in  #
    #              the Atacama Cosmology Telescope.  #
    #                                                #
    # Parameters:                                    #
    #             Signal  : Signal Strength (Jy);    #
    #               Freq  : Frequency (GHz);         #
    #              Omega  : Solid Angle (sr);        #
    #------------------------------------------------#
    
    """
    
    # Boltzmann Constant:
    k_B = codata.value('Boltzmann constant')
    
    # Computes the radiation Wavelength (in meters):
    Lambda = 3e8/(Freq*1e9)
    
    # Jansky to J/m² Convertion:
    Jy_to_Jpm2 = 1e-26
    
    return ((Lambda**2)*Signal*Jy_to_Jpm2)/(2*k_B*Omega)


def Detector_Noise(Duration):
    """
    #------------------------------------------------#
    # Description: Computes the detector noise level #
    #              for a signal of given duration.   #
    #                                                #
    # Parameters:                                    #
    #             Duration  : Signal duration (ms);  #
    #------------------------------------------------#
    
    """
    
    # Average Detector Noise (in K√s):
    Av_Det_Noise = 400e-6
    
    # Computes the Noise Level:
    Noise_Level = Av_Det_Noise/np.sqrt(Duration/1000.)
    
    return Noise_Level

    
def Solid_Angle(Pol_Int, Az_Int, Beam):
    """
    #------------------------------------------------#
    # Description: Computes the instrument's beam    #
    #              solid angle integral for a given  #
    #              range of angles in spherical      #
    #              coordinates. 'Pol_Int' as well as #
    #              'Az_Int' must be a lists with two #
    #              elements each, determining the    #
    #              initial and final angles of       #
    #              integration for those variables.  #
    #              The 'Beam' must be an array of    #
    #              dimension 2 encapsulating the     #
    #              beam profile.                     #
    #                                                #
    # Parameters:                                    #
    #             Pol_Int  : Polar Angles (degrees); #
    #              Az_Int  : Azimuthal Angles (rad); #
    #                Beam  : Beam Profile;           #
    #------------------------------------------------#
    
    """
    
    # Converts the Beam Range to Radians:
    Range = Beam[0]*np.pi/180.
    
    # Computes the Azimuthal Integral:
    Phi_Partial = np.abs(Az_Int[1] - Az_Int[0])
    
    # Constructs a Filter Imposing the Polar Integration Limits:
    Filter = np.logical_and(Beam[0] > Pol_Int[0], Beam[0] < Pol_Int[1])
    
    # Computes the Polar Integration:
    Theta_Partial = np.trapz(Beam[1][Filter]*np.sin(Range[Filter]), Range[Filter])
    
    return Phi_Partial*Theta_Partial


In [ ]:
print 'PA3 (90 GHz):'
omega_tot = Solid_Angle([0, 0.9/2.], [0, np.pi*2.], Beam_PA3_90)
omega_part = Solid_Angle([0.00, 0.055/2.], [0, 2*np.pi], Beam_PA3_90)
omega_ratio = omega_part/omega_tot
print(omega_ratio)
print 'Total Solid Angle:', Solid_Angle([0, 0.9/2.], [0, 2*np.pi], Beam_PA3_90), 'sr'




a = 0.055
print 'Ratio of beam profile at angular separation of', a, '=', beam_profile(a)

In [ ]:
pr = PixelReader()
pg = PlotGlitches(tod_key="tod_data", cosig_key="cuts")
cs = cuts['coincident_signals']
#event = [203586, 203591, 5, 3] 
#event = [217833, 217837, 4, 4]
event = [49567, 49572, 5, 3]
stime = event[0]
etime = event[1]
pixels = pixels_affected_in_event(cs,event)

In [ ]:
# Signal Range in Jansky:
Signal_Range = np.linspace(0, 100, 1000)

# Duration Range in Milliseconds:
Duration_Range = np.linspace(1, 10, 5)

plt.figure(figsize=(12,8))

print pixels
print 'XY (and pythagorean) Coordinates of Pixels in Degrees:'

distance = [1.,2.,3.,4.,5.,6.,7.,8.,9.,10.]
l = random.choice(distance)
print 'Distance of signal 1/', l
print '\n'
for i,j in enumerate(pixels):
    x = sky_x[pixels[i]]
    y = sky_y[pixels[i]]
    z = .055 #dist between adjacent detectors
    omega_tot = Solid_Angle([0, 0.9/l], [0, 2*np.pi], Beam_PA3_90)
    omega_part = Solid_Angle([0.00, z/l], [0, 2*np.pi], Beam_PA3_90)
    omega_ratio = omega_part/omega_tot
    plt.plot(Signal_Range, Brightness_Temperature(Signal_Range, 90, 5.18162844559e-07)*omega_ratio/2.,'--', label=str(j))
    print j, ':', x, y
    print 'Omega Ratio:', omega_ratio
    print '\n'

plt.hlines(Detector_Noise(2.5), Signal_Range.min(), Signal_Range.max(), alpha=0.5, linestyle='dotted', label='Noise Level (2.5 ms)')    
#plt.hlines(Detector_Noise(7.5), Signal_Range.min(), Signal_Range.max(), alpha=0.5, linestyle='dashed', label='T = 7.5 ms')
#plt.hlines(Detector_Noise(15), Signal_Range.min(), Signal_Range.max(), alpha=0.5, linestyle='solid', label='T = 15 ms')

plt.yscale('log')
plt.xlim([0, 30])

plt.xlabel('FRB Flux Density, S (Jy)')
plt.ylabel('Signal (K)')

plt.title ('PA3 (90 GHz)')
plt.legend(title='Detector ID',loc=4)
plt.show()

In [ ]:
def timeseries(pixel_id, s_time, e_time, buffer=10):
    start_time = s_time - buffer
    end_time = e_time + buffer
    a1, a2 = pr.get_f1(pixel_id)
    d1, d2 = tod_data.data[a1], tod_data.data[a2]
    # try to remove the mean from start_time to end_time                                                                                                     
    d1 -= np.mean(d1[start_time:end_time])

    time = tod_data.ctime - tod_data.ctime[0]
    time = time[start_time:end_time]

    d_1 = d1[start_time:end_time]

    return time, d_1

In [ ]:
plt.figure(figsize=(10,8))
pr.plot(pixels)
plt.title('Pixels Affected')
plt.show()

In [ ]:
for i in pixels:
    j = pr.get_pixels_within_radius(i,1)
    x, y = timeseries(i,stime,etime)
    base_sig = max(y)
    max_sig = []
    for n in j:
        a,b = timeseries(n,stime,etime)
        max_sig.append(max(b))
    ratios = np.asarray(max_sig)
    ratios/base_sig
    print 'Pixel Affected:', i 
    print 'Adjacent Pixels:', j
    print 'Ratios of adjacent signals:', ratios
    plt.subplot(121)
    pr.plot(i)
    
    plt.subplot(122)
    pr.plot(j)
    plt.show()
    
#x, y = timeseries(527,stime,etime)
#plt.plot(x,y)
#plt.show()

In [ ]:
from scipy.optimize import curve_fit
Beam_PA3_90 = np.loadtxt('beam_profile_170112_2015_pa3_90GHz_instant.txt', unpack=True, delimiter='  ', skiprows=4)

In [ ]:
#use 271 for first event 
pid = pixels[0]

x, y = timeseries(pid,stime,etime)
y += abs(min(y))
mean = sum(x * y) / sum(y)
sigma = np.sqrt(sum(y * (x - mean)**2) / sum(y))

def Gauss(x, a, x0, sigma):
    return a * np.exp(-(x - x0)**2 / (2 * sigma**2))

popt,pcov = curve_fit(Gauss, x, y, p0=[max(y), mean, sigma])
y2 = Gauss(x, *popt)
normalized = (y2-min(y2))/(max(y2)-min(y2))
beam_x = np.linspace(-0.5,0.5,len(normalized))

plt.figure(figsize=(12,16))
plt.subplot(311)
plt.title('Glitch Fit')
plt.plot(x,y,label="Raw")
plt.plot(x, y2, 'r-', label='Fit')
plt.legend()

plt.subplot(312)
plt.plot(x,normalized,label="Normalized")
plt.title('Normalized')
plt.legend()


plt.subplot(313)
plt.title('Beam Comparison')
plt.plot(Beam_PA3_90[0], Beam_PA3_90[1], color='magenta', linewidth=2)
plt.plot(-Beam_PA3_90[0], Beam_PA3_90[1], color='magenta', linewidth=2,label="Beam PA3")
plt.plot(beam_x,normalized, label="Normalized")
plt.legend()
plt.show()